## **PCR Primer Design using Python**
**Written by Kilian Zindel**

Polymerase Chain Reaction (PCR) is a method used to amplify (or make billions of copies of) a DNA strand. It is often a precursor to many other procedures used in genetic testing and research. In this method, DNA is replicated using repeated cycles of heating and cooling, called thermal cycles. There are three steps in each cycle:

1. **Denaturation**: The temperature is raised to near boiling (~95°C), causing the double-stranded DNA to separate (or denature) into single strands.
2. **Annealing**: The temperature is decreased to around 50–65°C, allowing **PCR primers** to bind (or anneal) to their complementary sequences on the DNA template.
3. **Extension**: The temperature is raised to around 68–72°C, where a DNA polymerase enzyme binds to the **primers** and adds nucleotides to the 3' end, extending the DNA strand.

This cycle is repeated multiple times, leading to exponential amplification of the target DNA. Under ideal conditions, 30 cycles can produce over a billion copies of the target DNA fragment.

**PCR Primers** are short, single-stranded DNA sequences, typically spanning 18 to 25 nucleotides in length. They are essential for the extension phase of PCR because the Polymerase cannot start a new sequence, only add to the 3' end of an existing one. There are different kinds of primers designed for different use cases. In this tutorial I will be designing primers optimized for [Polymerase Chain Reaction (PCR)](https://www.biointeractive.org/classroom-resources/polymerase-chain-reaction-pcr). 

In [1]:
# The power of exponential growth 
number_of_cycles = 30
duplicate_DNA_strands = pow(2, number_of_cycles)
duplicate_DNA_strands

1073741824

![PCR-image](https://upload.wikimedia.org/wikipedia/commons/thumb/a/ab/Polymerase_chain_reaction-en.svg/1024px-Polymerase_chain_reaction-en.svg.png)

#### Calculating the Melting Temperature (Tm) of a Primer

An important consideration when designing a PCR primer is its melting temperature (Tm). During the annealing process, the temperature plays a critical role in how primers bind to the template DNA, as it affects the stability of hydrogen bonds between the primer and template. If the temperature is too high (e.g., >72°C), the hydrogen bonds break, and the primer begins to dissociate from the DNA. Conversely, if the temperature is too low (e.g., <50°C), primers are more likely to bind to and stick to non-target sequences.

The temperature range of 50–65°C is considered optimal for the annealing temperature (Ta) in PCR. The exact annealing temperature should be set 3–5°C below the primer's melting temperature. This means the optimal melting temperature for the primer should be 55°C or higher.

This temperature range is optimal for two main reasons:

1. Specificity: The temperature is high enough for primers to anneal specifically to their complementary sequence on the template DNA.
2. Stability: The temperature is low enough for primers to form stable bonds with the template DNA.

The **melting temperature (Tm)** depends on three main factors:

1. GC Content: Higher GC content leads to a higher melting temperature.
2. Length: Longer primers have a higher melting temperature.
3. Ionic Strength of the DNA Solution (total ion concentration): DNA is more stable in a solution with higher ionic strength.

Studies have shown a linear relationship between primer GC content and melting temperature. The primer's melting temperature can be approximated using the **Wallace rule**. Below, we'll define two functions to calculate GC content and melting temperature.


In [2]:
def calc_gc_content(seq):
    # calculate the percentage of nucleotides in sequence that are either 'G' or 'C'
    c = seq.count('C')
    g = seq.count('G')
    gc_content = (g + c) / len(seq)
    return round(100 * gc_content, 0) 

def calc_tm(seq):
    # calculate the melting temperature of the sequence
    # Simple approximation (Wallace rule):
    # Tm ≈ 2°C*(A+T) + 4°C*(G+C)
    a = seq.count('A')
    t = seq.count('T')
    g = seq.count('G')
    c = seq.count('C')
    return 2*(a+t) + 4*(g+c)

#### Detecting runs of repeated nucleotides
We will need a function to detect runs of four or more identical bases. Avoiding such primers is considered best practice because excessive repeated bases can cause "breathing" of the primer, where a base bulges out. For example, a run of five guanine nucleotides might bind imperfectly to a run of four cytosine nucleotides, potentially leading to mispriming.

In [3]:
def check_max_run(seq, max_run=3):
    count = 1
    for i in range(1, len(seq)):
        # check if current and next base are the same
        if seq[i-1] == seq[i]:
            # if so: increment the count
            count += 1
            if count > max_run:
                return True
        else:
            # otherwise: reset the count
            count = 1 
    return False

#### Checking the GC Clamp

Because guanine (G) and cytosine (C) form three hydrogen bonds—compared to the two bonds formed by adenine (A) and thymine (T)—GC bonds provide greater stability for the primer-template bond. Having 1–3 G/C nucleotides at the 3' end of the primer helps to stabilize primer binding during PCR, reduces the likelihood of the primer dissociating prematurely, and helps facilitate extension by Taq polymerase.

The 3' end of the primer, which I will define as the last five nucleotides, is especially important because this is where Taq polymerase begins synthesizing the new DNA strand. A stable 3' end improves the likilhood of a successful replication. 

However, we also need to ensure that we don't overclamping—have more than three G/C nucleotides in the last five bases—which can cause problems. Too much GC content at the 3' end increases the likelihood of primers forming secondary structures, for example: 

1. Primer dimers: Two primers bind to each other instead of the template DNA
2. Hairpins: These are formed when the primer folds back on itself, creating a loop locked in by internal base pairing. 

In order to get an optimal primer, let's write a function which checks that the 3' end contains 1–3 G/C nucleotides and check whether the final nucleotide is a G or C.

In [4]:
def check_gc_clamp(seq, max_gc_clamp=3):
    # check the last nucleotide 
    last_nt = seq[-1]
    if not (last_nt == 'G' or last_nt == 'C'):
        return False
    
    # check the GC clamp 
    clamp = seq[-5:]
    g = clamp.count('G')
    c = clamp.count('C')
    gc_content = g + c
    if 1 <= gc_content <= max_gc_clamp:
        # The GC Clamp is optimal 
        return True 
    else: 
        return False 

#### Best Practices in Selecting Primers
- https://www.bocsci.com/resources/what-are-oligonucleotide-primers.html?srsltid=AfmBOoqrulSfsaXCtnkj0b8bwHnZRzN-KGALtljojbyG2wuYoq10FRNd
- https://www.thermofisher.com/blog/behindthebench/pcr-primer-design-tips/

There are a number of guidelines and best practices we should adhere to when selecting Primers
- Primers typically range from 18 to 24 nucleotides in length
- Optimal melting temperature ranges from 55°C to 65°C
- Optimal GC content is between 40% and 60%
- A stable 3' end which terminates in 2-3 G/C bases.
- Avoid runs of more than 4 identical bases

#### Now Let's go get some Sequences!
Here are some websites with the info we need:
- https://www.ncbi.nlm.nih.gov/gene
- https://www.uniprot.org/uniprotkb/P25084/entry

Instead of manually getting sequences on the webiste, we can use the python request library and the NCBI API to retrieve the information we need. 

In [5]:
import requests

def fetch_sequence(accession):
    # gets FASTA data from the NCBI database using an accession number 
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"
    params = {
        'db': 'nucleotide',
        'id': accession,
        'rettype': 'fasta',
        'retmode': 'text'
    }
    response = requests.get(base_url + "efetch.fcgi", params=params)
    
    # parse FASTA to extract sequence
    fasta_data = response.text 
    # remove the first line and all newline characters
    lines = fasta_data.splitlines()
    sequence = "".join(lines[1:])
    return sequence

# Accession Number for Example Sequence: 
accession = "NM_001268006.2" # Caenorhabditis elegans act-1 gene,

# Fetch Sequence from NCBI Database 
seq = fetch_sequence(accession)
seq

'TTTCATATGTTTTCGTCATAAATAAATAGTTACAAGAAATAATGGAGTCGTCTGACAATTTACATGATATAGATAATCTTGAAAACGGTAACATGGCTTGCCAGTGCTTCTTGGTTGGAGCCGGATACGTGGCTCTTGCAGCTGTGGCTTATCGTCTTTTGACGATTTTCTCGAATATTTTGGGCCCATACGTTCTTCTGTCGCCAATCGATTTGAAGAAAAGAGCTGGAGCTTCTTGGGCTGTTGTCACCGGAGCCACTGACGGAATCGGAAAAGCATACGCCTTCGAATTGGCTCGTCGTGGATTCAATGTCCTGCTCGTTTCGCGTACCCAATCAAAACTCGATGAGACGAAGAAGGAGATTCTCGAGAAGTATTCCAGCATTGAGGTCCGCACTGCCGCCTTCGACTTCACCAACGCTGCTCCTTCTGCTTACAAAGATCTTCTCGCCACCTTGAACCAAGTAGAGATCGGAGTTCTTATTAACAACGTTGGAATGAGCTACGAATATCCAGATGTACTTCACAAAGTTGACGGTGGAATCGAGCGTCTTGCAAACATCACCACCATCAACACTCTTCCACCAACATTGCTCTCCGCCGGAATCCTTCCACAAATGGTCGCACGAAAGGCTGGAGTCATTGTTAATGTTGGATCTTCAGCTGGAGCAAATCAAATGGCTCTCTGGGCTGTGTATTCAGCTACAAAGAAGTATGTCTCCTGGCTCACCGCTATCCTCCGAAAAGAATATGAACATCAAGGAATCACTGTCCAAACTATTGCTCCAATGATGGTCGCCACAAAGATGTCAAAAGTCAAGAGAACTTCATTCTTCACTCCAGACGGAGCCGTGTTCGCTAAATCAGCTCTGAACACTGTTGGAAATACCTCAGACACCACCGGATACATCACGCATCAACTTCAACTCGAGCTCATGGATCTCATTCCAACATTCATCCGCGACAAGATCCTCACAAATATGAGTGTCGGAACTCGTG

#### Generating Candidates for Primers
Let's set some parameters that we can use for our primer. We can define a ranges for the length, gc content, melting temperature etc. Then if our code doesn't generate any primers for the given sequence, we can go back and loosen the requirements. 

The following script uses a sliding window to iterate through every possible primer sequence of various lengths, then checks if each potential primer meets the constraints we've defined (optimal temperature, good clamp etc...)

In [6]:
import pandas as pd

sequence = "ATGGCCTTGGTTGACGGTTTTCTTGAGCTGGAACGCTCAAGTGGAAAATTGGAGTGGAGCGCCATCCTGCAGAAGATGGCGAGCGACCTTGGATTCTCGAAGATCCTGTTCGGCCTGTTGCCTAAGGACAGCCAGGACTACGAGAACGCCTTCATCGTCGGCAACTACCCGGCCGCCTGGCGCGAGCATTACGACCGGGCTGGCTACGCGCGGGTCGACCCGACGGTCAGTCACTGTACCCAGAGCGTACTGCCGATTTTCTGGGAACCGTCCATCTACCAGACGCGAAAGCAGCACGAGTTCTTCGAGGAAGCCTCGGCCGCCGGCCTGGTGTATGGGCTGACCATGCCGCTGCATGGTGCTCGCGGCGAACTCGGCGCGCTGAGCCTCAGCGTGGAAGCGGAAAACCGGGCCGAGGCCAACCGTTTCATGGAGTCGGTCCTGCCGACCCTGTGGATGCTCAAGGACTACGCACTGCAGAGCGGTGCCGGACTGGCCTTCGAACATCCGGTCAGCAAACCGGTGGTTCTGACCAGCCGGGAGAAGGAAGTGTTGCAGTGGTGCGCCATCGGCAAGACCAGTTGGGAGATATCGGTTATCTGCAACTGCTCGGAAGCCAATGTGAACTTCCATATGGGAAATATTCGGCGGAAGTTCGGTGTGACCTCCCGCCGCGTAGCGGCCATTATGGCCGTTAATTTGGGTCTTATTACTCTCTGA"
# Set Paramaters
min_length = 18
max_length = 24
gc_range = (40, 60)  # GC content range in %
tm_range = (57, 62)  # Melting temperature range in °C
max_self_complementarity = 5  # Maximum allowed score for self-complementarity
max_gc_clamp = 3  # Max GC bases at the 3' end to avoid "GC clamp"
max_hairpin_delta_g = -9.0  # Threshold for hairpin stability (kcal/mol)
target = (50,250) # target 

# Generate Primer Candidates 
max_length = 24
candidates = [] 
for length in range(min_length, max_length+1):
    for start in range(target[0]-length):
        primer = sequence[start:start+length]
        
        # check primer to ensure it meets constraints
        if not check_gc_clamp(primer):
            continue 
        
        if check_max_run(primer):
            continue 

        gc = calc_gc_content(primer)
        if not (gc > gc_range[0] and gc < gc_range[1]):
            continue 
        
        tm = calc_tm(primer)
        if not (tm > tm_range[0] and tm < tm_range[1]):
            continue

        candidates.append({
            'start': start,
            'length': length,
            'sequence': primer,
            'GC%': gc,
            'Tm': tm,
        })

df = pd.DataFrame(candidates)
df      

,start,length,sequence,GC%,Tm
0,22,19,TTGAGCTGGAACGCTCAAG,53.0,58
1,24,19,GAGCTGGAACGCTCAAGTG,58.0,60
2,25,19,AGCTGGAACGCTCAAGTGG,58.0,60


#### Forward and Reverse Primers

To amplify a DNA sequence in PCR, two primers are needed. A forward primer, to bind to the template strand, and a reverse primer, to bind to the complementary strand. The two primers should flank the target sequence on either side, with the 3' ends facing eachother.

![primer_and_target_image](https://pressbooks.bccampus.ca/kathleef/wp-content/uploads/sites/1093/2020/08/Primers-on-target-DNA-png-4-1024x309.png)
Designing the reverse primer requires an extra step. The reverse primer should match the sequence complementary to the one we are using to design it, so we will need to take the reverse complement of the primer when we test it. Let's write a function to do this.

In [7]:
def reverse_complement(seq):    
    complement_map = {'A':'T','T':'A','C':'G','G':'C','N':'N'}
    rc = ''.join(complement_map.get(base, 'N') for base in reversed(seq))
    return rc

To ensure the forward primer is positioned as close as possible to the beginning of the target sequence, the script starts at the beginning of the sequence and iterates backwards in the 5' direction. For the reverse primer, the script starts at the end of the target region and moves forward in the 3' direction, but once a primer is identified, its reverse complement is calculated and evaluated against the design constraints.

The script is similar to the previous one but incorporates these directional sliding windows. It first identifies forward primers, storing the best candidates in a list. Then it searches for reverse primers, repeating the same process.

A sliding window is used to iterate through every possible primer surrounding the target region. Each potential primer is checked to ensure it complies with our constraints, such as GC content, melting temperature, etc. Valid primers are stored in a data struture.

In [8]:
sequence = "ATGGCCTTGGTTGACGGTTTTCTTGAGCTGGAACGCTCAAGTGGAAAATTGGAGTGGAGCGCCATCCTGCAGAAGATGGCGAGCGACCTTGGATTCTCGAAGATCCTGTTCGGCCTGTTGCCTAAGGACAGCCAGGACTACGAGAACGCCTTCATCGTCGGCAACTACCCGGCCGCCTGGCGCGAGCATTACGACCGGGCTGGCTACGCGCGGGTCGACCCGACGGTCAGTCACTGTACCCAGAGCGTACTGCCGATTTTCTGGGAACCGTCCATCTACCAGACGCGAAAGCAGCACGAGTTCTTCGAGGAAGCCTCGGCCGCCGGCCTGGTGTATGGGCTGACCATGCCGCTGCATGGTGCTCGCGGCGAACTCGGCGCGCTGAGCCTCAGCGTGGAAGCGGAAAACCGGGCCGAGGCCAACCGTTTCATGGAGTCGGTCCTGCCGACCCTGTGGATGCTCAAGGACTACGCACTGCAGAGCGGTGCCGGACTGGCCTTCGAACATCCGGTCAGCAAACCGGTGGTTCTGACCAGCCGGGAGAAGGAAGTGTTGCAGTGGTGCGCCATCGGCAAGACCAGTTGGGAGATATCGGTTATCTGCAACTGCTCGGAAGCCAATGTGAACTTCCATATGGGAAATATTCGGCGGAAGTTCGGTGTGACCTCCCGCCGCGTAGCGGCCATTATGGCCGTTAATTTGGGTCTTATTACTCTCTGA"
min_length=18
max_length=24
gc_range = (40, 60)
tm_range = (57, 62)
max_self_complementary = 5
max_gc_clamp = 3
target = (50,250)

primer_candidates = [] 
nt = target[0]

while (len(primer_candidates) < 10 and nt > 0):
    for length in range(min_length, max_length):
        primer_start = nt - length - 1
        if primer_start < 0:
            continue 
        primer = sequence[primer_start:primer_start + length]
        
        # check primer to ensure it meets constraints
        if not check_gc_clamp(primer):
            continue 
        if check_max_run(primer):
            continue 
        gc = calc_gc_content(primer)
        if not (gc > gc_range[0] and gc < gc_range[1]):
            continue     
        tm = calc_tm(primer)
        if not (tm > tm_range[0] and tm < tm_range[1]):
            continue
        primer_candidates.append({
            'start': primer_start,
            'type': 'forward',
            'length': length,
            'sequence': primer,
            'GC%': gc,
            'Tm': tm,
        })
    nt -= 1

reverse_primer_candidates = [] 
nt = target[1]

while (len(primer_candidates) < 20 and nt < len(sequence)):
    for length in range(min_length, max_length):
        primer_start = nt
        if primer_start > len(sequence) - length - 1:
            continue 
        primer = sequence[primer_start:primer_start + length]
        primer = reverse_complement(primer) 
        # check primer to ensure it meets constraints
        if not check_gc_clamp(primer):
            continue 
        if check_max_run(primer):
            continue 
        gc = calc_gc_content(primer)
        if not (gc > gc_range[0] and gc < gc_range[1]):
            continue     
        tm = calc_tm(primer)
        if not (tm > tm_range[0] and tm < tm_range[1]):
            continue
        primer_candidates.append({
            'start': primer_start,
            'type': 'reverse',
            'length': length,
            'sequence': primer,
            'GC%': gc,
            'Tm': tm,
        })
    nt += 1

df = pd.DataFrame(primer_candidates)
df = df.sort_values(by='Tm') # sort by termpature
df

,start,type,length,sequence,GC%,Tm
2,22,forward,19,TTGAGCTGGAACGCTCAAG,53.0,58
5,264,reverse,19,TCTGGTAGATGGACGGTTC,53.0,58
7,271,reverse,19,TTTCGCGTCTGGTAGATGG,53.0,58
8,272,reverse,19,CTTTCGCGTCTGGTAGATG,53.0,58
11,285,reverse,19,AGAACTCGTGCTGCTTTCG,53.0,58
15,294,reverse,19,CTTCCTCGAAGAACTCGTG,53.0,58
17,297,reverse,19,AGGCTTCCTCGAAGAACTC,53.0,58
3,262,reverse,19,TGGTAGATGGACGGTTCCC,58.0,60
6,269,reverse,19,TCGCGTCTGGTAGATGGAC,58.0,60
4,263,reverse,19,CTGGTAGATGGACGGTTCC,58.0,60


#### Choosing our Primer

From the list of candidates, we can now select our forward and reverse primers. The melting temperatures (Tm) of the forward and reverse primers should be similar, ideally within 2–3°C of each other. To make this easier, the list has been sorted by melting temperature. Based on this criterion, Primer 1 and Primer 4 seem like strong candidates, they have the same melting temperature and both tightly flank the target region:

- **Possible Forward Primer: GAGCTGGAACGCTCAAGTG**
- **Possible Reverse Primer: CTGGTAGATGGACGGTTCC**

While this program generates working primers, there are a couple ways it could be improved. A robust primer design tool should also check for secondary structures, such as:

1. Hairpins: Structures formed when primers fold back on themselves.
2. Primer Dimers: Primers bind to each other instead of the template DNA.

Detecting these issues requires more sophisticated algorithms. Most Primer Design tools use complex thermodynamic models to more accurately calculate melting temperature (Tm) and predict secondary structures. These models account for interactions between bases and other factors affecting primer stability.

Finally, the last step in primer design is ensuring the primer is specific. This requires checking that primers do not bind to unintended other of the genome, which could happen if the exact sequence (or a similar one) is repeated elsewhere. Specificity checks are performed using a BLAST (Basic Local Alignment Search Tool) search against the target genome. This step helps to avoid off-target amplification and ensures the primers are unique to the desired region. The NIH website has a free to use BLAST tool that can search their database.

- https://blast.ncbi.nlm.nih.gov/Blast.cgi

Overal, this project implements the basic principles of primer design for PCR using Python, providing a script that can generate and evaluate potential primers based on criteria such as desired melting temperature, gc content etc. This project introduced me to the complexities of primer design and helped me better understand the value of computation in biology. 